In [ ]:
import torch
import torch.nn as nn
from transformers import AutoProcessor, LlavaForConditionalGeneration, Trainer, TrainingArguments, BitsAndBytesConfig
from PIL import Image
import json
import numpy as np
import os
from pathlib import Path
from typing import List, Dict, Optional, Tuple
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")